## Importing relevant libraries

In [1]:
import numpy as np
import scipy.linalg as la
import scipy.spatial as spat
from scipy.stats import unitary_group
from scipy.stats import moment
from scipy.stats import skew, kurtosis
from scipy.optimize import curve_fit
from scipy.linalg import norm
import matplotlib.pyplot as plt
import math
from dataclasses import dataclass

# Libraries for implementing the VQD algorithm
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector
from qiskit.primitives import Sampler, Estimator
from qiskit_aer import AerSimulator
from qiskit_algorithms.utils import algorithm_globals
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Session, Options
from qiskit_ibm_runtime import Estimator as EstimatorNew
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import RealAmplitudes, TwoLocal, EfficientSU2
from qiskit_algorithms.optimizers import *
from qiskit_algorithms.state_fidelities import ComputeUncompute

from qiskit_algorithms.eigensolvers import EigensolverResult, VQD
from qiskit_algorithms import NumPyMinimumEigensolver, VQE

# Import classical optimizers
from qiskit_algorithms.optimizers import SPSA, P_BFGS, COBYLA, IMFIL, SNOBFIT, NELDER_MEAD, SLSQP, NFT, ADAM, POWELL, GradientDescent, BOBYQA

# Import Statevector and SparsePauliOp
from qiskit.quantum_info import SparsePauliOp, Statevector

# Import noise models
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)

# Import a fake backend and Qiskit simulators and/or noise libraries
from qiskit_aer import AerSimulator
# from qiskit_aer.primitives import Estimator as AerEstimator 
# from qiskit_aer.noise import NoiseModel

# Import the FakeManila backend
from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeMontrealV2, FakeGuadalupeV2, FakeManila

## Constructing the FPE operator

In [3]:
## Classical implementation
"""
Purpose: 
    Find the analytical solution given the input parameters
Input: 
    x: array representing the x coordinates over which the solution is to be found
    a, Γ: parameters for the Ornstein-Uhlenbeck equation
Output:
    y: array of the value of the analytical solution over x
"""

def OrnsteinUhlenbeck(x, a = 1, gamma = 1):
    y = np.sqrt(a/(2*np.pi*gamma)) * np.exp((-a * x ** 2)/(2*gamma))
    
    return y

"""
Purpose:
    Calculate the weighted Hermite polynomials using recursive relations up to H_n
Input:
    n: the highest number of Hermite polynomials to be evaluated at
    x: the value of x at which the weighted Hermite polynomials are to be evaluated
Output:
    Hermite: the value of the weighted Hermite polynomials
Example: HermiteSeries(2, 4) -> [H_0(4), H_1(4), H_2(4)]
"""

def HermiteSeries(n, x):
    Hermite = np.zeros((1, n+1))
    Hermite[0][0] = 1.0 * np.exp(-x**2/2)
    Hermite[0][1] = 2.0 * x *np.exp(-x**2/2)
    
    for ni in range(1, n):
        Hermite[0][ni+1] = 2*x*Hermite[0][ni] - 2*ni*Hermite[0][ni-1]
    
    return Hermite

"""
Purpose:
    Reconstruct the functions represented by the coefficients of weighted Hermite polynomials in spatial coordinates
Input:
    x: array containing the x values over which the function is to be evaluated
    coefficients: the coefficients corresponding to the Hermite polynomials (should be zero-mode of the operator)
    nmax: the highest order of weighted Hermite polynomials used
    s: the scaling factor
Output:
    y: the reconstruction
"""

def project(x, coefficients, nmax, s):
    
    y = np.zeros(x.shape)
    
    for i in (range(len(x))):
        x0 = s*x[i]
        hermite_values = HermiteSeries(nmax, x0)
        y[i] += np.dot(hermite_values, coefficients)
        
    return (x, y)

"""
Purpose:
    Normalize the PDF so that the area under the curve is 1
Input:
    x, y: arrays representing the unnormalized
Output:
    (x, y): normalized PDF
"""

def normalize(x, y):
    dx = x[1] - x[0]
    sum = 0
    
    for i in range(len(x)):
        sum = sum + (y[i] * dx)
    
    y = y/sum
    
    return (x, y)

"""
Purpose:
    Calculate the RMS
Input:
    y_op, y: the y values generated by the operator and the analytical solution
Output:
    RMS
"""

def rms_diff(y_op, y):
    return np.sqrt(np.sum((y_op-y)**2)/len(y))

"""
Purpose:
    Find s given a and Γ such that the projection in Hermite space is just H_0
Input:
    a, Γ: parameters for Ornstein-Uhlenbeck
Output:
    s
"""

def special_s(a, gamma):
    return np.sqrt(a/gamma)

def euler_representation(z):
    A = abs(z)
    phase = math.atan(z.imag/z.real)
    
    return A, phase

def generate_positive_semidefinite_matrix(eigenvalues):
    ## Function to transform the FPE operator to the Hermite basis and return a finite truncated matrix for the ...
    ## ... FPE operator
    
    n = len(eigenvalues)
    s = np.diag(eigenvalues)
    q, _ = la.qr(np.random.rand(n, n))
    semidef = np.dot(np.transpose(q), s)
    semidef = np.dot(semidef, q)
    
    return semidef

def normalize_probability(y, dx):
    ## Function to return a normalized PDF 
    
    total = np.sum(y) * dx
    y = y / total
    
    return y

def perturbed_ornstein_uhlenbeck(x0, a = 1, gamma = 1, c = 0, shift = 0):
    ## Function to compute the exact solution to the perturbed Ornstein-Uhlenbeck equation
    
    assert len(x0) > 1, "Error: x0 should be a vector"

    x = x0 - shift
    y = np.exp(-a*x**2 /(2*gamma) - c*x**4 /(4*gamma))
    dx = x[2] - x[1]
    
    y = normalize_probability(y, dx)
    
    return y

def special_L(a, gamma):
    ## Function to compute the characteristic length scale
    
    return np.sqrt(a/gamma)

def delta(N, n):
    ## Function to implement the Kronecker delta function
    
    if N == n:
        return 1
    else:
        return 0
    
def matrix_element(N, n, L):
    # Function to compute the matrix elements of the position operator
    
    value = L/np.sqrt(2) * (np.sqrt(n) * delta(N, n-1) + np.sqrt(n+1) * delta(N, n+1))
    
    return value

def create_position(nmax, L):
    # Function to construct the position operator
    
    op = np.zeros((nmax + 1, nmax + 1))
    
    for N in range(nmax+1):
        row = N
        for n in range(nmax+1):
            col = n
            op[row, col] = matrix_element(N, n, L)
            
    return op

def f(N,n):
    return 0.5*( np.sqrt(n*(n-1))*delta(N,n-2) + delta(N,n) - np.sqrt((n+1)*(n+2))*delta(N,n+2))

def g(N,n, L):
    return (0.5 / L**2)*( np.sqrt(n*(n-1))*delta(N,n-2) - (2*n+1)*delta(N,n) + np.sqrt((n+1)*(n+2))*delta(N,n+2))

def t(N, n, L):
    val1 = np.sqrt(n*(n-1)*(n-2)*(n-3))*delta(N, n-4)
    val2 = (2*n+2)*np.sqrt(n*(n-1))*delta(N, n-2)
    val3 = (6*n+3)*delta(N, n)
    val4 = -2*n*np.sqrt((n+1)*(n+2))*delta(N, n+2)
    val5 = -1*np.sqrt((n+1)*(n+2)*(n+3)*(n+4))*delta(N, n+4)

    return (L**2/4)*( val1 + val2 + val3 + val4 + val5 )

def element_perturbed(N, n, L, a, c, gamma):
    ## Function to compute the matrix elements of the finite truncated matrix for the FPE operator (in the Hermite basis)
    
    return -(a*f(N,n) + c*t(N, n, L) + gamma*g(N, n, L))

def create_operator_perturbed(nmax, L, a, c, gamma):
    ## Function to generate the FPE operator matrix in the Hermite basis
    op = np.zeros((nmax+1, nmax+1))
    
    for N in range(nmax+1):
        row = N
        for n in range(nmax+1):
            col = n
            op[row, col] = element_perturbed(N, n, L, a, c, gamma)
    
    """
    if np.any(np.nan, op):
        print("There's an nan in the operator")
    elif np.any(np.inf, op):
        print("There is an inf in the operator")
    """

    return op

def state_n(nmax, x0, L):
    ## Function to compute the Hermite polynomials upto the order specified by nmax
    
    assert L > 0, "Error (state_n): input L must be greater than or equal to 0"

    states = np.zeros(nmax + 1)
    x = x0 / L

    states[0] = np.sqrt(L / np.sqrt(1/np.pi))* 1.0 * np.exp(-x**2/2)

    if nmax > 0:
        states[1] = np.sqrt(L / np.sqrt(1/np.pi))* (1/np.sqrt(2)) * 2.0 * x * np.exp(-(x**2)/2)
    
    for ni in range(2, nmax + 1):
        states[ni] = ((np.sqrt(2)*x) / np.sqrt(ni))*states[ni-1] - (ni-1)/np.sqrt((ni)*(ni-1))*states[ni-2]

    return states

def integrate_eigenvector(x0, y, nmax, L):
    ## Function to compute the coefficients corresponding to the calculated Hermite polynomials by numerical integration
    ## Note: here, we use the left-hand integration method
    
    dx = x0[2] - x0[1]
    eigenvector = np.zeros(nmax + 1)

    for i in range(len(x)):

        states = state_n(nmax, x0[i], L)
        states = states * y[i] * dx

        eigenvector = eigenvector + states
        
    return {"eigenvector" : eigenvector, "dx" : dx, "nmax" : nmax, "x0" : x0, "L" : L}

def make_operator_even(op):
    op_new = np.zeros((op.shape[0]//2, op.shape[1]//2))
    
    for row in range(op_new.shape[0]):
        for col in range(op_new.shape[1]):
            op_new[row, col] = op[row*2, col * 2]
    
    return op_new

def reconstruct_eigenvector(cache, normalize = True, only_even = False):
    ## Function to reconstruct the PDF using the cache obtained previously
    
    eigenvector = cache["eigenvector"]
    nmax = cache["nmax"]
    x0 = cache["x0"]
    dx = cache["dx"]
    L = cache["L"]
    
    if not only_even:
        eigenvector = cache["eigenvector"]
    else:
        eigenvector_old = cache["eigenvector"]
        eigenvector = np.zeros(nmax + 1)
        
        for i in range(len(eigenvector_old)):
            eigenvector[2*i] = eigenvector_old[i]

    y = np.zeros(len(x0))

    for i in range(len(x0)):
        states = state_n(nmax, x0[i], L)
        y[i] += (np.dot(states, eigenvector))
    if normalize:   
        y = normalize_probability(y, dx)
        
    return (x0, y)

def find_zeromode(op_nonhermitian, nmax, x0, dx, L, which = "single", only_even = False):
    ## Function to compute the zeromode, its position, and the eigenvalue corresponding to the zeromode
    
    assert which == "nonhermitian" or which == "single", "Error: currently only supports which = {\"nonhermitian\", \"single\"}"
    
    if only_even:
        op_processed = make_operator_even(op_nonhermitian)
    else:
        op_processed = op_nonhermitian
        
    if which == "nonhermitian":
        op = op_processed
    elif which == "single":
        op = np.dot(np.transpose(op_processed), op_processed)

    eigenvalues, eigenvectors = la.eig(op)

    index = np.argmin(np.real(eigenvalues)**2)

    min_eigenval = eigenvalues[index]
    
    zeromode = eigenvectors[:, index]

    return {"operator" : op, "which" : which, \
            "eigenvector" : np.real(zeromode), "index" : index, "eigenvalue" : np.real(min_eigenval), \
            "nmax" : nmax, "x0" : x0, "dx" : dx, "L" : L, "spectrum" : eigenvalues, "P" : eigenvectors}

def rms_diff(y0, y, sigdigits = 0):
    ## Function to compute the RMS difference between the analytically computed PDF and the one ...
    ## ... obtained using the FPE operator
    
    assert len(y0) == len(y), "Error: the length of y0 and y should be the same"
    diff = np.sum((y0-y)**2)/len(y0)
    
    return np.sqrt(diff)

def analyze_collapse(initial_state, P, index):
    num_basis = P.shape[1]
    coefficients = np.zeros((num_basis, ))

    for i in range(num_basis):
        basis = P[:, i]
        basis = normalize_eigenvector(basis)

        coefficients[i] = np.dot(basis, initial_state)

    normalized_coefficients = normalize_eigenvector(coefficients)

    reconstruction = np.zeros((P.shape[0], ))

    for i in range(num_basis):
        basis = P[:, i]
        basis = normalize_eigenvector(basis)

        added_vector = coefficients[i] * basis
        reconstruction = reconstruction + added_vector

    probability = normalized_coefficients ** 2
    
    probability_zeromode = probability[index]
    assert np.sum(probability) - 1 < 0.00001, "Error: unity normalization of probability is not observed"

    print("The initial_state is: ")
    print(initial_state)
    print()

    print("The coefficient expansion is: ")
    print(coefficients)
    print()

    print("The normalized coefficient expansion is: ")
    print(normalized_coefficients)
    print()

    print("The reconstructed vector is: ")
    print(reconstruction)
    print()

    print("The probability of collapsing into each eigenvector is: ")
    print(probability)
    print()

    print("The probability of collapsing into the zeromode is: ")
    print(probability_zeromode)
    print()
    
    return probability_zeromode, probability

def get_unitary(matrix, add_half = False):
    """
    Purpose: given a matrix, returns the unitary, hermitian matrix to be diagonalized
    Input: matrix -> the matrix to be diagonalized
    Output: U -> the unitary matrix
            nqubits -> the number of qubis needed to represent the basis of U
            dimension -> the dimension of the original matrix
    """
    assert matrix.ndim == 2, "Error: Only a matrix maybe processed"
    assert matrix.shape[0] == matrix.shape[1], "Error: Only a square matrix maybe processed"

    if np.any(np.transpose(matrix) != matrix):
        matrix_T = np.transpose(matrix)
        matrix = np.dot(matrix_T, matrix)

    ## Finding the dimension of the matrix
    dimension_hermitian = matrix.shape[0]

    ## Finding the number of qubits required to represent the matrix
    nqubits = int(np.ceil(np.log2(dimension_hermitian)))

    ## Construct the relevant matrix
    op_dim = 2 ** nqubits
    op = np.eye(op_dim)
    op[0:dimension_hermitian, 0:dimension_hermitian] = np.copy(matrix)

    if add_half:
        op = op + np.pi * np.eye(op.shape[0])

    U = la.expm(1j*op)

    # Get the dimensions of the unitary matrix
    dimension = U.shape[0]

    return U, nqubits, dimension

def expect_value(zeromode, matrix):
    
    value = np.dot(matrix, zeromode)
    expect = np.dot(np.transpose(zeromode), value)
    
    return expect

## Helper functions

In [6]:
def find_probability(eigenvector_raw):
    """
    Purpose: Find the probability associated with each basis of an eigenvector
    Input: eigenvector_raw -> Numpy array documenting the number of times each basis is detected within the eigenvector
    Output: eigenvector_prob -> Numpy array documenting the probability of detecting each basis
    """
    count_total = np.sum(eigenvector_raw)
    eigenvector_prob = eigenvector_raw / count_total
    
    return eigenvector_prob

def find_amplitude(eigenvector_prob):
    """
    Purpose: Finding the probability amplitude of each basis using quantum mechanics
    Input: eigenvector_prob -> Numpy array documenting the probability that each basis is measured
    Output: eigenvector -> Numpy array representing the eigenvector
    """
    eigenvector = np.sqrt(eigenvector_prob)
    return eigenvector

def normalize_eigenvector(vector):
    """
    Purpose: Normalizes a vector such that its norm is 1
    Input: vector -> The vector to be normalized
    Output: vector -> The normalized vector
    """
    L2 = np.sum(np.square(vector))
    vector = vector / np.sqrt(L2)

    return vector

def make_operator_even(op):
    op_new = np.zeros((op.shape[0]//2, op.shape[1]//2))

    for row in range(op_new.shape[0]):
        for col in range(op_new.shape[1]):
            op_new[row, col] = op[row*2, col * 2]

    return op_new

def get_pdf(n, x, dx, L, shift, zeromode_qpe, normalize = True, make_even = False):
    # Function to construct the ground state PDF using the VQSVD zeromode
    
    if not make_even:
        eigenvector = zeromode_qpe
    else:
        eigenvector_old = zeromode_qpe
        eigenvector = np.zeros(n + 1)
        for i in range(len(eigenvector_old)):
            eigenvector[2*i] = eigenvector_old[i]
            
    x0 = x - shift
    
    # Computing the PDF
    y = np.zeros(len(x0))

    for i in range(len(x0)):
        states = state_n(nmax, x0[i], L)
        y[i] += (np.dot(states, eigenvector))
    
    if normalize:
        y = normalize_probability(y, dx)

    return x0, y

def compute_expectation_x_squared_simpson(x, y, n):
    """
    Computes the expectation value of x^2 using Simpson's rule for numerical integration.
    
    Parameters:
    x (array-like): Discrete values of x.
    y (array-like): Corresponding values of the probability density function (PDF) at x.
    
    Returns:
    float: The expectation value of x^2.
    """
    # Ensure x and y are numpy arrays
    x = np.array(x)
    y = np.array(y)
    
    # Compute x^2
    x_squared = x**n
    
    # Check if the number of intervals is even, if not make it even by truncating the last point
    if len(x) % 2 == 0:
        x = x[:-1]
        y = y[:-1]
        x_squared = x_squared[:-1]
    
    # Compute the integral using Simpson's rule
    h = (x[-1] - x[0]) / (len(x) - 1)
    integral = y[0] * x_squared[0] + y[-1] * x_squared[-1] + \
               4 * np.sum(y[1:-1:2] * x_squared[1:-1:2]) + \
               2 * np.sum(y[2:-2:2] * x_squared[2:-2:2])
    integral *= h / 3
    
    return integral

def get_pdf(n, x, dx, L, shift, zeromode_qpe, normalize = True, make_even = False):
    # Function to construct the ground state PDF using the VQSVD zeromode
    
    if not make_even:
        eigenvector = zeromode_qpe
    else:
        eigenvector_old = zeromode_qpe
        eigenvector = np.zeros(n + 1)
        for i in range(len(eigenvector_old)):
            eigenvector[2*i] = eigenvector_old[i]
            
    x0 = x - shift
    
    # Computing the PDF
    y = np.zeros(len(x0))

    for i in range(len(x0)):
        states = state_n(nmax, x0[i], L)
        y[i] += (np.dot(states, eigenvector))
    
    if normalize:
        y = normalize_probability(y, dx)

    return x0, y

# Fidelity measure 1
def get_fidelity(zeromode_classic, zeromode_vqe):
    # Function to compute the infidelity

    overlap = np.dot(np.transpose(zeromode_vqe), zeromode_classic)
    fidelity = 1 - overlap ** 2
    return fidelity

# Fidelity measure 2
def get_similarity(a, b):
    # Function to compute the similarity between 2 zeromodes
    
    numerator = np.abs(np.dot(a.conj().T, b))**2
    denominator = np.linalg.norm(a)**2 * np.linalg.norm(b)**2
    
    return numerator / denominator

def compute_errors(expect_classical, expect_quantum):

    error = np.abs(expect_classical - expect_quantum) / expect_classical
    return error

## VQE implementation

In [8]:
# VQE run for a given ansatz and optimizer
def run_vqe_ansatz_analysis(matrix, ansatz, optimizer, seed, exact_ground_state):

    # Get the Pauli-decomposed form of the operator
    qub_hamiltonian = SparsePauliOp.from_operator(matrix)
    dimension = matrix.shape[0]
    num_qubits = int(np.log2(dimension))

    # Set up the random initial point
    np.random.seed(seed)
    initial_point = np.random.uniform(-np.pi, np.pi, ansatz.num_parameters)

    # Initialize the Estimator primitive
    estimator = Estimator()

    # Logging class for VQE callback
    @dataclass
    class VQELog:
        parameters: list
        values: list
        def update(self, count, parameters, mean, _metadata):
            self.values.append(mean)
            self.parameters.append(parameters)
            
    log = VQELog([], [])
    
    # Run VQE with the given ansatz and optimizer
    vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point, callback=log.update)

    # Get the VQE results
    result = vqe.compute_minimum_eigenvalue(qub_hamiltonian)

    # Get the number of optimizer function calls
    num_calls = result.cost_function_evals

    # Extract the optimal parameters and construct the state vector
    optimal_params = result.optimal_point
    final_circuit = ansatz.assign_parameters(optimal_params)
    vqe_statevector = Statevector.from_instruction(final_circuit)

     # Convert the quantum and classical zeromodes into 4 x 1 arrays
    exact_ground_state = np.array(exact_ground_state).reshape((len(exact_ground_state), 1))
    vqe_statevector = vqe_statevector.data.tolist()

    if len(exact_ground_state) == 6:
        vqe_statevector = vqe_statevector[:6]
        zeromode = np.array(vqe_statevector).reshape((len(exact_ground_state), 1))
        zeromode = np.real(zeromode)
    
        # Compute the fidelity measure
        fidelity_value = get_similarity(exact_ground_state, zeromode)
    
    else:
        zeromode = np.array(vqe_statevector).reshape((len(exact_ground_state), 1))
        zeromode = np.real(zeromode)
    
        # Compute the fidelity measure
        fidelity_value = get_similarity(exact_ground_state, zeromode)
    
    return zeromode, fidelity_value, num_calls

## Compute the matrix and the zeromode

In [12]:
def get_zeromode(nmax, a, c, L, gamma):
    # Function to construct the matrix and get the zeromode

    #3, 5, 1, 1
    dx = 0.01
    x = np.linspace(-4, 4, int((8)/dx))
    
    ## Finding the zeromode through diagonalization
    op_nonhermitian = create_operator_perturbed(nmax, L, a, c, gamma)
    only_even = True
    
    # Matrix
    cache_diagonalization = find_zeromode(op_nonhermitian, nmax, x, dx, L, which = "single", only_even = only_even)
    matrix = cache_diagonalization['operator']

    # Get the classical zeromode
    A, P = la.eig(matrix)
    
    # Get the zeromode
    A_real = np.real(A)
    index = np.where(A_real == np.amin(A_real))[0][0]
    
    eigenvalue = A[index]
    zeromode_classic = P[:, index]
    
    zeromode_classic = np.real(normalize_eigenvector(zeromode_classic))
    zeromode_classic = np.reshape(zeromode_classic, (zeromode_classic.size, 1))

    return matrix, zeromode_classic

### Evaluating performance metrics at multiple ansatz depths

In [15]:
def analyze_ansatz_performance(matrix, zeromode_classic, optimizer, num_qubits, max_depth):

    # Initialize dictionaries to store results
    zeromodes = {}
    metrics = {}  # To store fidelity and function call counts

    optimizer_name = optimizer.__class__.__name__

    # Loop through each ansatz type
    for AnsatzClass in [RealAmplitudes, TwoLocal, EfficientSU2]:
        # Initialize ansatz with appropriate configurations
        if AnsatzClass == RealAmplitudes:
            ansatz = AnsatzClass(num_qubits=num_qubits, entanglement='full', reps=1)
        elif AnsatzClass == TwoLocal:
            ansatz = AnsatzClass(num_qubits=num_qubits, rotation_blocks=['ry'], entanglement_blocks='cx', reps=1)
        elif AnsatzClass == EfficientSU2:
            ansatz = AnsatzClass(num_qubits=num_qubits, su2_gates=['ry'], entanglement='sca', reps=1)

        ansatz_name = AnsatzClass.__name__
        pair_name = f"{optimizer_name}-{ansatz_name}"
        print(f"\nRunning VQE for optimizer-ansatz pair: {pair_name}")

        # Initialize dictionaries to store depth-specific results
        zeromodes[pair_name] = {}
        metrics[pair_name] = {}

        for current_depth in range(1, max_depth + 1):
            # Set `reps` (depth) for the current ansatz
            ansatz.reps = current_depth

            # Temporary storage for the fidelity results
            fidelities = []
            function_calls = []  # To track the number of function calls for each run
            all_zeromodes = []  # To store all zeromodes for the depth

            # Perform multiple independent VQE runs to calculate fidelity and resource usage
            for run in range(6):  # Number of independent runs
                seed = run + 1  # Set a unique seed for each run

                # Run VQE
                zeromode, fidelity_value, function_call_count = run_vqe_ansatz_analysis(
                    matrix=matrix, ansatz=ansatz, optimizer=optimizer, seed=seed,
                    exact_ground_state=zeromode_classic)

                # Append results
                fidelities.append(fidelity_value)
                function_calls.append(function_call_count)
                all_zeromodes.append(zeromode)

            # Identify the run with the highest fidelity at this depth
            best_run_index = np.argmax(fidelities)
            best_fidelity = fidelities[best_run_index]
            best_zeromode = all_zeromodes[best_run_index]
            best_function_calls = function_calls[best_run_index]

            # Store results for the current depth
            zeromodes[pair_name][current_depth] = best_zeromode
            metrics[pair_name][current_depth] = {
                'fidelity': best_fidelity,
                'function_calls': best_function_calls
            }

            print(f"{pair_name} - Depth {current_depth}: Best fidelity = {best_fidelity}, Function calls = {best_function_calls}, \
            Best zeromode: {best_zeromode}")

    return zeromodes, metrics

### Quantum (VQE) analysis

### For a = 1 (N = 2) -- Non-Gaussian case

### Get the matrix and the zeromode

In [18]:
## Get the matrix and zeromode
nmax = 3
a = 1
gamma = 1
dx = 0.01
c = 2
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode_classic = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode_classic)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode_classic, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[  5.765625   -21.34578596]
 [-21.34578596  79.265625  ]]

The zeromode is:
[[-0.96558703]
 [-0.26008015]]

The value of <x^2> is:
0.41050077276583463



C:\Users\Owner\AppData\Local\Temp\ipykernel_2916\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


### Run VQE for multiple ansatz depths (fix at 5000 optimization iterations) -- P_BFGS optimizer

In [27]:
## Get the optimizers
maxiter = 5000
optimizer = P_BFGS(maxfun = maxiter)

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
max_depth = 6

# Compute 
zeromodes_N_2, metrics_N_2 = analyze_ansatz_performance(matrix, zeromode_classic, optimizer, num_qubits, max_depth)

For Windows, using only current process. Multiple core use not supported.



Running VQE for optimizer-ansatz pair: P_BFGS-RealAmplitudes


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-RealAmplitudes - Depth 1: Best fidelity = [[1.]], Function calls = 21,             Best zeromode: [[0.96558703]
 [0.26008014]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


...

P_BFGS-EfficientSU2 - Depth 4: Best fidelity = [[1.]], Function calls = 66,             Best zeromode: [[-0.96558703]
 [-0.26008014]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 5: Best fidelity = [[1.]], Function calls = 49,             Best zeromode: [[0.96558703]
 [0.26008015]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 6: Best fidelity = [[1.]], Function calls = 56,             Best zeromode: [[-0.96558703]
 [-0.26008015]]


### Print the results

In [29]:
# Print the results
print('The best zeromodes are:')
print(zeromodes_N_2)
print()

print('The performance metrics are:')
print(metrics_N_2)
print()

The best zeromodes are:
{'P_BFGS-RealAmplitudes': {1: array([[0.96558703],
       [0.26008014]]), 2: array([[-0.96558704],
       [-0.26008014]]), 3: array([[0.96558703],
       [0.26008014]]), 4: array([[-0.96558703],
       [-0.26008014]]), 5: array([[0.96558703],
       [0.26008015]]), 6: array([[-0.96558703],
       [-0.26008015]])}, 'P_BFGS-TwoLocal': {1: array([[-0.96558703],
       [-0.26008014]]), 2: array([[0.96558703],
       [0.26008014]]), 3: array([[0.96558703],
       [0.26008015]]), 4: array([[-0.96558704],
       [-0.26008013]]), 5: array([[0.96558703],
       [0.26008015]]), 6: array([[0.96558703],
       [0.26008015]])}, 'P_BFGS-EfficientSU2': {1: array([[0.96558703],
       [0.26008014]]), 2: array([[-0.96558704],
       [-0.26008014]]), 3: array([[0.96558703],
       [0.26008014]]), 4: array([[-0.96558703],
       [-0.26008014]]), 5: array([[0.96558703],
       [0.26008015]]), 6: array([[-0.96558703],
       [-0.26008015]])}}

The performance metrics are:
{'P_BFGS-R

### Computing relative errors in <x^2>

In [31]:
# Get the zeromodes
zeromodes = [
    zeromode
    for depths in zeromodes_N_2.values()
    for zeromode in depths.values()
]

# Initialize
relative_errors = []

# Compute 
for zeromode in zeromodes:
    x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
    
    # Ensure quantum_expectation is a scalar
    quantum_expectation = compute_expectation_x_squared_simpson(x, y, 2)
    if isinstance(quantum_expectation, np.ndarray):
        quantum_expectation = quantum_expectation.item()  # Convert to scalar if necessary
    
    # Compute the relative error
    relative_error = compute_errors(classical_expectation, quantum_expectation)
    relative_errors.append(relative_error)

# Get the relative errors for the individual optimizers
errors_realamps = relative_errors[:6]
errors_twolocal = relative_errors[6:12]
errors_efficientsu2 = relative_errors[12:18]

## Print the relative errors
print('The errors for RealAmplitudes are:')
print(errors_realamps)
print()

print('The errors for TwoLocal are:')
print(errors_twolocal)
print()

print('The errors for EfficientSU2 are:')
print(errors_efficientsu2)
print()

C:\Users\Owner\AppData\Local\Temp\ipykernel_9592\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


The errors for RealAmplitudes are:
[6.485065525859509e-09, 1.175356352773887e-08, 6.340211173568491e-09, 5.440752904338527e-09, 7.781326223080094e-09, 1.3829953216531535e-09]

The errors for TwoLocal are:
[7.518450111549826e-09, 1.0326035770413547e-08, 6.520989354381997e-09, 1.6450794440302076e-08, 1.1239354762612961e-09, 7.184408432425778e-09]

The errors for EfficientSU2 are:
[6.485065525859509e-09, 1.175356352773887e-08, 6.340211173568491e-09, 5.440752904338527e-09, 7.781326223080094e-09, 1.3829953216531535e-09]



### For N = 4

### Get the matrix and the zeromode

In [69]:
## Get the matrix and zeromode
nmax = 7
a = 1
gamma = 1
dx = 0.01
c = 2
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode_classic = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode_classic)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode_classic, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[   6.140625    -23.4671063    35.51760127   -9.22378041]
 [ -23.4671063    96.890625   -151.66269884  111.47028752]
 [  35.51760127 -151.66269884  376.140625   -354.47919269]
 [  -9.22378041  111.47028752 -354.47919269  972.890625  ]]

The zeromode is:
[[-0.97335411]
 [-0.22648723]
 [ 0.02489903]
 [ 0.02579447]]

The value of <x^2> is:
0.32868055910006333



C:\Users\Owner\AppData\Local\Temp\ipykernel_9592\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


### Run VQE for multiple ansatz depths (fix at 5000 optimization iterations)

In [71]:
## Get the optimizers
maxiter = 5000
optimizer = P_BFGS(maxfun = maxiter)

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
max_depth = 6

# Compute 
zeromodes_N_4, metrics_N_4 = analyze_ansatz_performance(matrix, zeromode_classic, optimizer, num_qubits, max_depth)

For Windows, using only current process. Multiple core use not supported.



Running VQE for optimizer-ansatz pair: P_BFGS-RealAmplitudes


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-RealAmplitudes - Depth 1: Best fidelity = [[1.]], Function calls = 185,             Best zeromode: [[ 0.97335408]
 [ 0.22648736]
 [-0.02489896]
 [-0.02579448]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


...

P_BFGS-EfficientSU2 - Depth 4: Best fidelity = [[1.]], Function calls = 935,             Best zeromode: [[ 0.97335411]
 [ 0.22648723]
 [-0.02489903]
 [-0.02579447]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 5: Best fidelity = [[1.]], Function calls = 611,             Best zeromode: [[-0.9733541 ]
 [-0.22648725]
 [ 0.02489902]
 [ 0.02579447]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 6: Best fidelity = [[1.]], Function calls = 660,             Best zeromode: [[-0.97335411]
 [-0.22648723]
 [ 0.02489904]
 [ 0.02579447]]


### Print the results

In [73]:
# Print the results
print('The best zeromodes are:')
print(zeromodes_N_4)
print()

print('The performance metrics are:')
print(metrics_N_4)
print()

The best zeromodes are:
{'P_BFGS-RealAmplitudes': {1: array([[ 0.97335408],
       [ 0.22648736],
       [-0.02489896],
       [-0.02579448]]), 2: array([[-0.97335411],
       [-0.22648724],
       [ 0.02489902],
       [ 0.02579445]]), 3: array([[ 0.97335411],
       [ 0.22648722],
       [-0.02489904],
       [-0.02579447]]), 4: array([[-0.97335411],
       [-0.22648723],
       [ 0.02489902],
       [ 0.02579446]]), 5: array([[ 0.97335411],
       [ 0.22648723],
       [-0.02489903],
       [-0.02579446]]), 6: array([[-0.97335411],
       [-0.22648721],
       [ 0.02489904],
       [ 0.02579446]])}, 'P_BFGS-TwoLocal': {1: array([[ 0.97335411],
       [ 0.22648721],
       [-0.02489903],
       [-0.02579445]]), 2: array([[ 0.9733541 ],
       [ 0.22648725],
       [-0.02489904],
       [-0.02579448]]), 3: array([[-0.97335411],
       [-0.22648724],
       [ 0.02489903],
       [ 0.02579446]]), 4: array([[-0.97335411],
       [-0.22648723],
       [ 0.02489903],
       [ 0.02579446]])

### Computing relative errors in <x^2>

In [75]:
# Get the zeromodes
zeromodes = [
    zeromode
    for depths in zeromodes_N_4.values()
    for zeromode in depths.values()
]

# Initialize
relative_errors = []

# Compute 
for zeromode in zeromodes:
    x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
    
    # Ensure quantum_expectation is a scalar
    quantum_expectation = compute_expectation_x_squared_simpson(x, y, 2)
    if isinstance(quantum_expectation, np.ndarray):
        quantum_expectation = quantum_expectation.item()  # Convert to scalar if necessary
    
    # Compute the relative error
    relative_error = compute_errors(classical_expectation, quantum_expectation)
    relative_errors.append(relative_error)

# Get the relative errors for the individual optimizers
errors_realamps = relative_errors[:6]
errors_twolocal = relative_errors[6:12]
errors_efficientsu2 = relative_errors[12:18]

## Print the relative errors
print('The errors for RealAmplitudes are:')
print(errors_realamps)
print()

print('The errors for TwoLocal are:')
print(errors_twolocal)
print()

print('The errors for EfficientSU2 are:')
print(errors_efficientsu2)
print()

C:\Users\Owner\AppData\Local\Temp\ipykernel_9592\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


The errors for RealAmplitudes are:
[4.225151467270674e-07, 1.2305205274640836e-07, 7.067765067259312e-08, 7.172066235568314e-08, 1.7222265177519086e-08, 3.237022767864794e-08]

The errors for TwoLocal are:
[7.341616759904789e-08, 1.414564862401792e-08, 4.7693864738350355e-08, 6.29983255889385e-08, 1.5628905649404547e-08, 3.442139737021861e-08]

The errors for EfficientSU2 are:
[4.225151467270674e-07, 7.830405261450795e-08, 6.17281161420991e-10, 1.1796256639449915e-09, 8.674358868405624e-08, 2.690135615342582e-08]



### For N = 8

### Get the matrix and the zeromode

In [77]:
## Get the matrix and zeromode
nmax = 15
a = 1
gamma = 1
dx = 0.01
c = 2
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode_classic = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode_classic)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode_classic, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[ 6.14062500e+00 -2.34671063e+01  3.55176013e+01 -9.22378041e+00
  -3.13747510e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.34671063e+01  9.68906250e+01 -1.51662699e+02  1.11470288e+02
  -6.65558976e+01 -2.10468228e+01  0.00000000e+00  0.00000000e+00]
 [ 3.55176013e+01 -1.51662699e+02  4.02390625e+02 -3.54479193e+02
   1.63951212e+02 -2.30876387e+02 -6.98044143e+01  0.00000000e+00]
 [-9.22378041e+00  1.11470288e+02 -3.54479193e+02  1.05164062e+03
  -5.71772019e+02  1.15363556e+02 -5.86246269e+02 -1.71932490e+02]
 [-3.13747510e+00 -6.65558976e+01  1.63951212e+02 -5.71772019e+02
   2.23364062e+03 -7.19516989e+02 -1.36244266e+02 -1.24067405e+03]
 [ 0.00000000e+00 -2.10468228e+01 -2.30876387e+02  1.15363556e+02
  -7.19516989e+02  4.19739062e+03 -6.89706553e+02 -7.16860080e+02]
 [ 0.00000000e+00  0.00000000e+00 -6.98044143e+01 -5.86246269e+02
  -1.36244266e+02 -6.89706553e+02  6.56939062e+03 -1.15978184e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.7

C:\Users\Owner\AppData\Local\Temp\ipykernel_9592\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


### Run VQE for multiple ansatz depths (fix at 5000 optimization iterations)

In [79]:
## Get the optimizers
maxiter = 5000
optimizer = P_BFGS(maxfun = maxiter)

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
max_depth = 6

# Compute 
zeromodes_N_8, metrics_N_8 = analyze_ansatz_performance(matrix, zeromode_classic, optimizer, num_qubits, max_depth)

For Windows, using only current process. Multiple core use not supported.



Running VQE for optimizer-ansatz pair: P_BFGS-RealAmplitudes


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-RealAmplitudes - Depth 1: Best fidelity = [[0.99999956]], Function calls = 546,             Best zeromode: [[ 9.70878537e-01]
 [ 2.38167165e-01]
 [-1.53123227e-02]
 [-2.01793370e-02]
 [ 5.01936006e-03]
 [ 1.26992339e-03]
 [-1.64603055e-03]
 [ 2.79435156e-04]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


...

P_BFGS-EfficientSU2 - Depth 4: Best fidelity = [[1.]], Function calls = 2048,             Best zeromode: [[ 9.70766013e-01]
 [ 2.38641228e-01]
 [-1.49874489e-02]
 [-2.02525309e-02]
 [ 4.87629712e-03]
 [ 1.51994334e-03]
 [-1.67451763e-03]
 [ 1.77600033e-04]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 5: Best fidelity = [[1.]], Function calls = 1444,             Best zeromode: [[-9.70765903e-01]
 [-2.38641712e-01]
 [ 1.49870880e-02]
 [ 2.02523165e-02]
 [-4.87637131e-03]
 [-1.52016465e-03]
 [ 1.67445998e-03]
 [-1.77623633e-04]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 6: Best fidelity = [[1.]], Function calls = 2134,             Best zeromode: [[ 9.70765844e-01]
 [ 2.38641929e-01]
 [-1.49872551e-02]
 [-2.02524678e-02]
 [ 4.87637256e-03]
 [ 1.51995647e-03]
 [-1.67449420e-03]
 [ 1.77606472e-04]]


### Print the results

In [81]:
# Print the results
print('The best zeromodes are:')
print(zeromodes_N_8)
print()

print('The performance metrics are:')
print(metrics_N_8)
print()

The best zeromodes are:
{'P_BFGS-RealAmplitudes': {1: array([[ 9.70878537e-01],
       [ 2.38167165e-01],
       [-1.53123227e-02],
       [-2.01793370e-02],
       [ 5.01936006e-03],
       [ 1.26992339e-03],
       [-1.64603055e-03],
       [ 2.79435156e-04]]), 2: array([[ 9.70765924e-01],
       [ 2.38641589e-01],
       [-1.49874261e-02],
       [-2.02525324e-02],
       [ 4.87633794e-03],
       [ 1.51995847e-03],
       [-1.67449242e-03],
       [ 1.77619786e-04]]), 3: array([[ 9.70767202e-01],
       [ 2.38635730e-01],
       [-1.49933433e-02],
       [-2.02567084e-02],
       [ 4.87319400e-03],
       [ 1.51835377e-03],
       [-1.67618943e-03],
       [ 1.74805648e-04]]), 4: array([[-9.70765882e-01],
       [-2.38641779e-01],
       [ 1.49872463e-02],
       [ 2.02524635e-02],
       [-4.87631614e-03],
       [-1.51995675e-03],
       [ 1.67449389e-03],
       [-1.77603403e-04]]), 5: array([[ 9.70765838e-01],
       [ 2.38641961e-01],
       [-1.49871534e-02],
       [-2.02524

### Computing relative errors in <x^2>

In [83]:
# Get the zeromodes
zeromodes = [
    zeromode
    for depths in zeromodes_N_8.values()
    for zeromode in depths.values()
]

# Initialize
relative_errors = []

# Compute 
for zeromode in zeromodes:
    x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
    
    # Ensure quantum_expectation is a scalar
    quantum_expectation = compute_expectation_x_squared_simpson(x, y, 2)
    if isinstance(quantum_expectation, np.ndarray):
        quantum_expectation = quantum_expectation.item()  # Convert to scalar if necessary
    
    # Compute the relative error
    relative_error = compute_errors(classical_expectation, quantum_expectation)
    relative_errors.append(relative_error)

# Get the relative errors for the individual optimizers
errors_realamps = relative_errors[:6]
errors_twolocal = relative_errors[6:12]
errors_efficientsu2 = relative_errors[12:18]

## Print the relative errors
print('The errors for RealAmplitudes are:')
print(errors_realamps)
print()

print('The errors for TwoLocal are:')
print(errors_twolocal)
print()

print('The errors for EfficientSU2 are:')
print(errors_efficientsu2)
print()

C:\Users\Owner\AppData\Local\Temp\ipykernel_9592\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


The errors for RealAmplitudes are:
[0.0011886153401521862, 9.60419259851094e-07, 0.00010253477062452442, 1.2739785156561975e-07, 6.346548687315207e-07, 5.886685884798525e-07]

The errors for TwoLocal are:
[0.005734534197608043, 2.095845992341826e-05, 4.2429600877841505e-07, 1.04917320298579e-06, 4.7438285084264516e-07, 9.770774650030607e-08]

The errors for EfficientSU2 are:
[0.005784641899194859, 1.3467830466538936e-06, 5.660459204391997e-07, 2.22291584963397e-06, 2.7024846517284423e-06, 3.739447975099702e-07]



### For N = 16

### Get the matrix and the zeromode

In [21]:
## Get the matrix and zeromode
nmax = 31
a = 1
gamma = 1
dx = 0.01
c = 2
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode_classic = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode_classic)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode_classic, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[ 6.14062500e+00 -2.34671063e+01  3.55176013e+01 -9.22378041e+00
  -3.13747510e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.34671063e+01  9.68906250e+01 -1.51662699e+02  1.11470288e+02
  -6.65558976e+01 -2.10468228e+01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 3.55176013e+01 -1.51662699e+02  4.02390625e+02 -3.54479193e+02
   1.63951212e+02 -2.30876387e+02 -6.98044143e+01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-9.22378041e+00  1.11470288e+02 -3.54479193e+02  1.05164062e+03
  -5.71772019e+02  1.15363556e+02 -5.86246269e+02 -1.71932490e+02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000

C:\Users\Owner\AppData\Local\Temp\ipykernel_26896\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


The value of <x^2> is:
0.3662333595845026



### Run VQE for multiple ansatz depths (fix at 50_000 optimization iterations)

### Note: here, we consider 6 independent VQE runs at each ansatz depth, not the usual 10

In [23]:
## Get the optimizers
maxiter = 50_000
optimizer = P_BFGS(maxfun = maxiter)

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
max_depth = 6

# Compute 
zeromodes_N_16, metrics_N_16 = analyze_ansatz_performance(matrix, zeromode_classic, optimizer, num_qubits, max_depth)

For Windows, using only current process. Multiple core use not supported.



Running VQE for optimizer-ansatz pair: P_BFGS-RealAmplitudes


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-RealAmplitudes - Depth 1: Best fidelity = [[0.99999859]], Function calls = 2340,             Best zeromode: [[ 9.70788348e-01]
 [ 2.38577751e-01]
 [-1.49342679e-02]
 [-1.99260662e-02]
 [ 5.15295422e-03]
 [ 1.30188077e-03]
 [-1.48348133e-03]
 [ 3.28807988e-04]
 [ 5.11227354e-05]
 [ 1.25637538e-05]
 [-7.86396632e-07]
 [-1.04934543e-06]
 [ 2.70545652e-07]
 [ 6.86271763e-08]
 [-8.87430272e-08]
 [ 2.06230648e-08]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


...

P_BFGS-EfficientSU2 - Depth 4: Best fidelity = [[0.99999988]], Function calls = 14343,             Best zeromode: [[-9.70630050e-01]
 [-2.39245635e-01]
 [ 1.44607940e-02]
 [ 1.99896505e-02]
 [-5.00400711e-03]
 [-1.65132274e-03]
 [ 1.51683339e-03]
 [-2.06291490e-04]
 [-1.15180176e-04]
 [ 5.09904724e-05]
 [ 3.43150374e-05]
 [-2.26807442e-05]
 [ 3.03267553e-05]
 [-4.32839221e-05]
 [ 2.54246574e-05]
 [ 7.64359463e-06]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 5: Best fidelity = [[1.]], Function calls = 14100,             Best zeromode: [[-9.70574250e-01]
 [-2.39486250e-01]
 [ 1.42931786e-02]
 [ 1.99079659e-02]
 [-5.09611981e-03]
 [-1.70866740e-03]
 [ 1.51666236e-03]
 [-2.71142632e-04]
 [-2.01186893e-04]
 [ 1.76769297e-04]
 [-4.34772836e-05]
 [-1.58394758e-05]
 [ 2.72879028e-05]
 [-9.75246751e-06]
 [ 2.65759363e-06]
 [ 4.25284669e-06]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 6: Best fidelity = [[1.]], Function calls = 10498,             Best zeromode: [[-9.70582738e-01]
 [-2.39450213e-01]
 [ 1.43159338e-02]
 [ 1.99105821e-02]
 [-5.09642302e-03]
 [-1.71631971e-03]
 [ 1.51617319e-03]
 [-2.72335431e-04]
 [-2.02397598e-04]
 [ 1.76310148e-04]
 [-4.39598916e-05]
 [-1.61985982e-05]
 [ 2.70054626e-05]
 [-9.77914272e-06]
 [ 2.58374283e-06]
 [ 4.25274354e-06]]


### Print the results

In [26]:
# Print the results
print('The best zeromodes are:')
print(zeromodes_N_16)
print()

print('The performance metrics are:')
print(metrics_N_16)
print()

The best zeromodes are:
{'P_BFGS-RealAmplitudes': {1: array([[ 9.70788348e-01],
       [ 2.38577751e-01],
       [-1.49342679e-02],
       [-1.99260662e-02],
       [ 5.15295422e-03],
       [ 1.30188077e-03],
       [-1.48348133e-03],
       [ 3.28807988e-04],
       [ 5.11227354e-05],
       [ 1.25637538e-05],
       [-7.86396632e-07],
       [-1.04934543e-06],
       [ 2.70545652e-07],
       [ 6.86271763e-08],
       [-8.87430272e-08],
       [ 2.06230648e-08]]), 2: array([[ 9.70601642e-01],
       [ 2.39373320e-01],
       [-1.43431714e-02],
       [-1.99181594e-02],
       [ 5.03776490e-03],
       [ 1.67037934e-03],
       [-1.47444354e-03],
       [ 2.35566158e-04],
       [ 9.15596320e-05],
       [ 2.29893655e-05],
       [-3.71404196e-06],
       [-1.43537878e-06],
       [ 1.20669059e-06],
       [ 6.02405975e-09],
       [ 1.53066376e-06],
       [-4.03490264e-07]]), 3: array([[-9.70519181e-01],
       [-2.39736730e-01],
       [ 1.40390063e-02],
       [ 1.97461588e-02],


### Computing relative errors in <x^2>

In [29]:
# Get the zeromodes
zeromodes = [
    zeromode
    for depths in zeromodes_N_16.values()
    for zeromode in depths.values()
]

# Initialize
relative_errors = []

# Compute 
for zeromode in zeromodes:
    x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
    
    # Ensure quantum_expectation is a scalar
    quantum_expectation = compute_expectation_x_squared_simpson(x, y, 2)
    if isinstance(quantum_expectation, np.ndarray):
        quantum_expectation = quantum_expectation.item()  # Convert to scalar if necessary
    
    # Compute the relative error
    relative_error = compute_errors(classical_expectation, quantum_expectation)
    relative_errors.append(relative_error)

# Get the relative errors for the individual optimizers
errors_realamps = relative_errors[:6]
errors_twolocal = relative_errors[6:12]
errors_efficientsu2 = relative_errors[12:18]

## Print the relative errors
print('The errors for RealAmplitudes are:')
print(errors_realamps)
print()

print('The errors for TwoLocal are:')
print(errors_twolocal)
print()

print('The errors for EfficientSU2 are:')
print(errors_efficientsu2)
print()

C:\Users\Owner\AppData\Local\Temp\ipykernel_26896\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


The errors for RealAmplitudes are:
[0.004516437336644318, 0.00017534659333234034, 0.004641073091126609, 0.00266606073429593, 0.0007136230363133777, 0.00026795419734732787]

The errors for TwoLocal are:
[0.0197145707889133, 0.0007380517572327267, 0.002427563696403335, 0.00022890975849587284, 0.00035482209496189547, 3.750991101606317e-05]

The errors for EfficientSU2 are:
[0.1053508087672761, 0.014404984786251901, 0.0033023422675540147, 0.002560448389651847, 0.00011858216161452855, 7.401820248463206e-05]



### For a = -1 (non-Gaussian) case 

### For N = 2

### Get the matrix and the zeromode

In [33]:
## Get the matrix and zeromode
nmax = 3
a = -1
gamma = 1
dx = 0.01
c = 2
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode_classic = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode_classic)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode_classic, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[ 17.015625   -37.25568853]
 [-37.25568853  84.515625  ]]

The zeromode is:
[[-0.91416021]
 [-0.40535307]]

The value of <x^2> is:
0.48931145833928297



C:\Users\Owner\AppData\Local\Temp\ipykernel_9592\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


### Run VQE for multiple ansatz depths (fix at 5000 optimization iterations)

In [36]:
## Get the optimizers
maxiter = 5000
optimizer = P_BFGS(maxfun = maxiter)

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
max_depth = 6

# Compute 
zeromodes_N_2, metrics_N_2 = analyze_ansatz_performance(matrix, zeromode_classic, optimizer, num_qubits, max_depth)

For Windows, using only current process. Multiple core use not supported.



Running VQE for optimizer-ansatz pair: P_BFGS-RealAmplitudes


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-RealAmplitudes - Depth 1: Best fidelity = [[1.]], Function calls = 18,             Best zeromode: [[0.91416021]
 [0.40535307]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


...

P_BFGS-EfficientSU2 - Depth 4: Best fidelity = [[1.]], Function calls = 36,             Best zeromode: [[0.9141602 ]
 [0.40535309]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 5: Best fidelity = [[1.]], Function calls = 63,             Best zeromode: [[-0.91416021]
 [-0.40535308]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 6: Best fidelity = [[1.]], Function calls = 72,             Best zeromode: [[0.91416021]
 [0.40535307]]


### Print the results

In [39]:
# Print the results
print('The best zeromodes are:')
print(zeromodes_N_2)
print()

print('The performance metrics are:')
print(metrics_N_2)
print()

The best zeromodes are:
{'P_BFGS-RealAmplitudes': {1: array([[0.91416021],
       [0.40535307]]), 2: array([[0.91416021],
       [0.40535307]]), 3: array([[0.91416021],
       [0.40535307]]), 4: array([[0.9141602 ],
       [0.40535309]]), 5: array([[-0.91416021],
       [-0.40535308]]), 6: array([[0.91416021],
       [0.40535307]])}, 'P_BFGS-TwoLocal': {1: array([[0.91416021],
       [0.40535307]]), 2: array([[0.91416021],
       [0.40535306]]), 3: array([[-0.91416021],
       [-0.40535307]]), 4: array([[0.91416021],
       [0.40535307]]), 5: array([[-0.91416021],
       [-0.40535307]]), 6: array([[0.91416021],
       [0.40535307]])}, 'P_BFGS-EfficientSU2': {1: array([[0.91416021],
       [0.40535307]]), 2: array([[0.91416021],
       [0.40535307]]), 3: array([[0.91416021],
       [0.40535307]]), 4: array([[0.9141602 ],
       [0.40535309]]), 5: array([[-0.91416021],
       [-0.40535308]]), 6: array([[0.91416021],
       [0.40535307]])}}

The performance metrics are:
{'P_BFGS-RealAmpli

### Computing relative errors in <x^2>

In [42]:
# Get the zeromodes
zeromodes = [
    zeromode
    for depths in zeromodes_N_2.values()
    for zeromode in depths.values()
]

# Initialize
relative_errors = []

# Compute 
for zeromode in zeromodes:
    x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
    
    # Ensure quantum_expectation is a scalar
    quantum_expectation = compute_expectation_x_squared_simpson(x, y, 2)
    if isinstance(quantum_expectation, np.ndarray):
        quantum_expectation = quantum_expectation.item()  # Convert to scalar if necessary
    
    # Compute the relative error
    relative_error = compute_errors(classical_expectation, quantum_expectation)
    relative_errors.append(relative_error)

# Get the relative errors for the individual optimizers
errors_realamps = relative_errors[:6]
errors_twolocal = relative_errors[6:12]
errors_efficientsu2 = relative_errors[12:18]

## Print the relative errors
print('The errors for RealAmplitudes are:')
print(errors_realamps)
print()

print('The errors for TwoLocal are:')
print(errors_twolocal)
print()

print('The errors for EfficientSU2 are:')
print(errors_efficientsu2)
print()

C:\Users\Owner\AppData\Local\Temp\ipykernel_9592\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


The errors for RealAmplitudes are:
[2.621293731546903e-09, 1.0017491779493877e-10, 2.1631113574015795e-09, 1.5468789817012852e-08, 4.015901544740527e-09, 2.8271304544424898e-09]

The errors for TwoLocal are:
[6.212846797494754e-10, 8.866759288450307e-09, 6.91177629301338e-10, 6.061814169145939e-09, 2.776463335652773e-09, 2.6125838013722766e-09]

The errors for EfficientSU2 are:
[2.621293731546903e-09, 1.0017491779493877e-10, 2.1631113574015795e-09, 1.5468789817012852e-08, 4.015901544740527e-09, 2.8271304544424898e-09]



### For N = 4

### Get the matrix and the zeromode

In [45]:
## Get the matrix and zeromode
nmax = 7
a = -1
gamma = 1
dx = 0.01
c = 2
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode_classic = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode_classic)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode_classic, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[  17.390625    -41.49832922   41.64132563   -2.51557647]
 [ -41.49832922  138.140625   -204.49024847  154.16103593]
 [  41.64132563 -204.49024847  387.390625   -424.31381877]
 [  -2.51557647  154.16103593 -424.31381877  790.140625  ]]

The zeromode is:
[[-0.80492107]
 [-0.55178885]
 [-0.21791911]
 [-0.01193297]]

The value of <x^2> is:
0.7580435055926756



C:\Users\Owner\AppData\Local\Temp\ipykernel_9592\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


### Run VQE for multiple ansatz depths (fix at 5000 optimization iterations)

In [48]:
## Get the optimizers
maxiter = 5000
optimizer = P_BFGS(maxfun = maxiter)

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
max_depth = 6

# Compute 
zeromodes_N_4, metrics_N_4 = analyze_ansatz_performance(matrix, zeromode_classic, optimizer, num_qubits, max_depth)

For Windows, using only current process. Multiple core use not supported.



Running VQE for optimizer-ansatz pair: P_BFGS-RealAmplitudes


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-RealAmplitudes - Depth 1: Best fidelity = [[1.]], Function calls = 140,             Best zeromode: [[0.80492106]
 [0.55178886]
 [0.21791912]
 [0.01193296]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


...

P_BFGS-EfficientSU2 - Depth 4: Best fidelity = [[1.]], Function calls = 220,             Best zeromode: [[-0.80492107]
 [-0.55178884]
 [-0.21791912]
 [-0.01193298]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 5: Best fidelity = [[1.]], Function calls = 1066,             Best zeromode: [[0.80492108]
 [0.55178883]
 [0.21791911]
 [0.01193297]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 6: Best fidelity = [[1.]], Function calls = 855,             Best zeromode: [[-0.80492108]
 [-0.55178884]
 [-0.2179191 ]
 [-0.01193296]]


### Print the results

In [51]:
# Print the results
print('The best zeromodes are:')
print(zeromodes_N_4)
print()

print('The performance metrics are:')
print(metrics_N_4)
print()

The best zeromodes are:
{'P_BFGS-RealAmplitudes': {1: array([[0.80492106],
       [0.55178886],
       [0.21791912],
       [0.01193296]]), 2: array([[-0.80492107],
       [-0.55178884],
       [-0.21791913],
       [-0.01193299]]), 3: array([[-0.80492107],
       [-0.55178884],
       [-0.21791912],
       [-0.01193298]]), 4: array([[0.80492107],
       [0.55178885],
       [0.21791911],
       [0.01193296]]), 5: array([[-0.80492109],
       [-0.55178883],
       [-0.2179191 ],
       [-0.01193297]]), 6: array([[-0.80492108],
       [-0.55178884],
       [-0.2179191 ],
       [-0.01193296]])}, 'P_BFGS-TwoLocal': {1: array([[-0.80492106],
       [-0.55178886],
       [-0.21791913],
       [-0.01193297]]), 2: array([[0.80492106],
       [0.55178885],
       [0.21791912],
       [0.01193298]]), 3: array([[-0.80492106],
       [-0.55178886],
       [-0.21791912],
       [-0.01193298]]), 4: array([[0.80492106],
       [0.55178886],
       [0.21791911],
       [0.01193297]]), 5: array([[0.8

### Computing relative errors in <x^2>

In [54]:
# Get the zeromodes
zeromodes = [
    zeromode
    for depths in zeromodes_N_4.values()
    for zeromode in depths.values()
]

# Initialize
relative_errors = []

# Compute 
for zeromode in zeromodes:
    x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
    
    # Ensure quantum_expectation is a scalar
    quantum_expectation = compute_expectation_x_squared_simpson(x, y, 2)
    if isinstance(quantum_expectation, np.ndarray):
        quantum_expectation = quantum_expectation.item()  # Convert to scalar if necessary
    
    # Compute the relative error
    relative_error = compute_errors(classical_expectation, quantum_expectation)
    relative_errors.append(relative_error)

# Get the relative errors for the individual optimizers
errors_realamps = relative_errors[:6]
errors_twolocal = relative_errors[6:12]
errors_efficientsu2 = relative_errors[12:18]

## Print the relative errors
print('The errors for RealAmplitudes are:')
print(errors_realamps)
print()

print('The errors for TwoLocal are:')
print(errors_twolocal)
print()

print('The errors for EfficientSU2 are:')
print(errors_efficientsu2)
print()

C:\Users\Owner\AppData\Local\Temp\ipykernel_9592\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


The errors for RealAmplitudes are:
[7.734090162796722e-09, 2.5709309418824114e-08, 2.061584945562113e-08, 2.283396093184703e-08, 2.836182549949179e-08, 2.850802088471765e-08]

The errors for TwoLocal are:
[1.9343140857494934e-08, 1.7204441068324432e-08, 1.9571378480080783e-08, 8.078578161128088e-09, 1.8729808157523082e-10, 5.106784086415227e-08]

The errors for EfficientSU2 are:
[7.734090162796722e-09, 1.913096334426826e-08, 1.2894078399654205e-08, 6.644435117635027e-09, 1.9414725042683576e-08, 3.51639706098326e-08]



### For N = 8

### Get the matrix and the zeromode

In [57]:
## Get the matrix and zeromode
nmax = 15
a = -1
gamma = 1
dx = 0.01
c = 2
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode_classic = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode_classic)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode_classic, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[ 1.73906250e+01 -4.14983292e+01  4.16413256e+01 -2.51557647e+00
  -3.13747510e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-4.14983292e+01  1.38140625e+02 -2.04490248e+02  1.54161036e+02
  -3.10594189e+01 -2.10468228e+01  0.00000000e+00  0.00000000e+00]
 [ 4.16413256e+01 -2.04490248e+02  4.13640625e+02 -4.62654398e+02
   2.97161572e+02 -1.33665277e+02 -6.98044143e+01  0.00000000e+00]
 [-2.51557647e+00  1.54161036e+02 -4.62654398e+02  9.24890625e+02
  -7.68209032e+02  4.17083625e+02 -3.82334523e+02 -1.71932490e+02]
 [-3.13747510e+00 -3.10594189e+01  2.97161572e+02 -7.68209032e+02
   1.81289062e+03 -1.04918443e+03  4.35981651e+02 -8.73066927e+02]
 [ 0.00000000e+00 -2.10468228e+01 -1.33665277e+02  4.17083625e+02
  -1.04918443e+03  3.27864062e+03 -1.20958947e+03  2.51869758e+02]
 [ 0.00000000e+00  0.00000000e+00 -6.98044143e+01 -3.82334523e+02
   4.35981651e+02 -1.20958947e+03  4.90064062e+03 -1.53414981e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.7

C:\Users\Owner\AppData\Local\Temp\ipykernel_9592\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


### Run VQE for multiple ansatz depths (fix at 5000 optimization iterations)

In [61]:
## Get the optimizers
maxiter = 5000
optimizer = P_BFGS(maxfun = maxiter)

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
max_depth = 6

# Compute 
zeromodes_N_8, metrics_N_8 = analyze_ansatz_performance(matrix, zeromode_classic, optimizer, num_qubits, max_depth)

For Windows, using only current process. Multiple core use not supported.



Running VQE for optimizer-ansatz pair: P_BFGS-RealAmplitudes


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-RealAmplitudes - Depth 1: Best fidelity = [[0.99787956]], Function calls = 693,             Best zeromode: [[ 8.49003239e-01]
 [ 5.01461167e-01]
 [ 1.66483775e-01]
 [ 3.00641137e-03]
 [-1.79539774e-04]
 [-2.95530975e-04]
 [ 1.78771164e-03]
 [ 9.98590563e-04]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


...

P_BFGS-EfficientSU2 - Depth 4: Best fidelity = [[1.]], Function calls = 3072,             Best zeromode: [[ 0.83477495]
 [ 0.5227743 ]
 [ 0.16895114]
 [-0.02397233]
 [-0.02597929]
 [ 0.00650642]
 [ 0.00351118]
 [-0.00300989]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 5: Best fidelity = [[1.]], Function calls = 4370,             Best zeromode: [[ 0.83477532]
 [ 0.52277385]
 [ 0.16895067]
 [-0.02397253]
 [-0.02597931]
 [ 0.00650642]
 [ 0.00351116]
 [-0.0030099 ]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 6: Best fidelity = [[1.]], Function calls = 2684,             Best zeromode: [[-0.83477498]
 [-0.52277424]
 [-0.16895119]
 [ 0.02397232]
 [ 0.02597934]
 [-0.00650642]
 [-0.00351119]
 [ 0.00300989]]


### Print the results

In [64]:
# Print the results
print('The best zeromodes are:')
print(zeromodes_N_8)
print()

print('The performance metrics are:')
print(metrics_N_8)
print()

The best zeromodes are:
{'P_BFGS-RealAmplitudes': {1: array([[ 8.49003239e-01],
       [ 5.01461167e-01],
       [ 1.66483775e-01],
       [ 3.00641137e-03],
       [-1.79539774e-04],
       [-2.95530975e-04],
       [ 1.78771164e-03],
       [ 9.98590563e-04]]), 2: array([[ 0.83477483],
       [ 0.52277444],
       [ 0.16895134],
       [-0.02397224],
       [-0.02597925],
       [ 0.00650641],
       [ 0.00351124],
       [-0.00300984]]), 3: array([[ 0.8347745 ],
       [ 0.52277482],
       [ 0.16895187],
       [-0.02397196],
       [-0.02597923],
       [ 0.00650647],
       [ 0.00351124],
       [-0.00300986]]), 4: array([[ 0.83477495],
       [ 0.52277433],
       [ 0.16895107],
       [-0.02397239],
       [-0.02597929],
       [ 0.00650643],
       [ 0.00351118],
       [-0.00300989]]), 5: array([[ 0.83477479],
       [ 0.52277451],
       [ 0.16895132],
       [-0.02397231],
       [-0.0259793 ],
       [ 0.00650643],
       [ 0.00351119],
       [-0.00300988]]), 6: array([[ 

### Computing relative errors in <x^2>

In [67]:
# Get the zeromodes
zeromodes = [
    zeromode
    for depths in zeromodes_N_8.values()
    for zeromode in depths.values()
]

# Initialize
relative_errors = []

# Compute 
for zeromode in zeromodes:
    x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
    
    # Ensure quantum_expectation is a scalar
    quantum_expectation = compute_expectation_x_squared_simpson(x, y, 2)
    if isinstance(quantum_expectation, np.ndarray):
        quantum_expectation = quantum_expectation.item()  # Convert to scalar if necessary
    
    # Compute the relative error
    relative_error = compute_errors(classical_expectation, quantum_expectation)
    relative_errors.append(relative_error)

# Get the relative errors for the individual optimizers
errors_realamps = relative_errors[:6]
errors_twolocal = relative_errors[6:12]
errors_efficientsu2 = relative_errors[12:18]

## Print the relative errors
print('The errors for RealAmplitudes are:')
print(errors_realamps)
print()

print('The errors for TwoLocal are:')
print(errors_twolocal)
print()

print('The errors for EfficientSU2 are:')
print(errors_efficientsu2)
print()

C:\Users\Owner\AppData\Local\Temp\ipykernel_9592\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


The errors for RealAmplitudes are:
[0.07535759628945446, 8.816414367922437e-07, 2.509566180484397e-06, 2.8526900864602765e-07, 4.065142500826371e-07, 4.987401485898919e-08]

The errors for TwoLocal are:
[0.0662106375650912, 2.411690472012576e-06, 2.408728760896542e-07, 1.7888004288335337e-07, 1.18520157221775e-07, 3.798767252648384e-07]

The errors for EfficientSU2 are:
[0.07535499695737244, 6.049491424651036e-07, 2.9053272202378656e-07, 1.0071471765125162e-07, 1.5840706221716522e-06, 1.410191432154513e-07]



### For N = 16

### Get the matrix and the zeromode

In [21]:
## Get the matrix and zeromode
nmax = 31
a = -1
gamma = 1
dx = 0.01
c = 2
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode_classic = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode_classic)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode_classic, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[ 1.73906250e+01 -4.14983292e+01  4.16413256e+01 -2.51557647e+00
  -3.13747510e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-4.14983292e+01  1.38140625e+02 -2.04490248e+02  1.54161036e+02
  -3.10594189e+01 -2.10468228e+01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 4.16413256e+01 -2.04490248e+02  4.13640625e+02 -4.62654398e+02
   2.97161572e+02 -1.33665277e+02 -6.98044143e+01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.51557647e+00  1.54161036e+02 -4.62654398e+02  9.24890625e+02
  -7.68209032e+02  4.17083625e+02 -3.82334523e+02 -1.71932490e+02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000

C:\Users\Owner\AppData\Local\Temp\ipykernel_2564\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


The value of <x^2> is:
0.6471064574160962



### Run VQE for multiple ansatz depths (fix at 50_000 optimization iterations)

### Note: 6 independent VQE runs at each ansatz depth considered

In [24]:
## Get the optimizers
maxiter = 50_000
optimizer = P_BFGS(maxfun = maxiter)

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
max_depth = 6

# Compute 
zeromodes_N_16, metrics_N_16 = analyze_ansatz_performance(matrix, zeromode_classic, optimizer, num_qubits, max_depth)

For Windows, using only current process. Multiple core use not supported.



Running VQE for optimizer-ansatz pair: P_BFGS-RealAmplitudes


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-RealAmplitudes - Depth 1: Best fidelity = [[0.99488686]], Function calls = 4041,             Best zeromode: [[ 8.68420693e-01]
 [ 4.76332550e-01]
 [ 1.35730048e-01]
 [-1.77158933e-02]
 [-1.27885846e-02]
 [-6.60396073e-03]
 [-1.03448866e-03]
 [-2.49227119e-03]
 [-1.18448929e-03]
 [-6.49739010e-04]
 [-1.85227417e-04]
 [ 2.44416497e-05]
 [ 1.68231990e-05]
 [ 5.87262866e-06]
 [-5.24921159e-06]
 [ 2.21507288e-05]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


...

P_BFGS-EfficientSU2 - Depth 4: Best fidelity = [[1.]], Function calls = 13713,             Best zeromode: [[ 8.33618514e-01]
 [ 5.24076210e-01]
 [ 1.70842838e-01]
 [-2.27842704e-02]
 [-2.54488525e-02]
 [ 6.89738970e-03]
 [ 3.84541518e-03]
 [-2.68990473e-03]
 [ 8.54561585e-05]
 [ 7.17013592e-04]
 [-3.42392207e-04]
 [ 3.99946898e-06]
 [ 1.31843946e-04]
 [-5.74639854e-05]
 [ 1.93769827e-05]
 [ 2.64105260e-05]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 5: Best fidelity = [[1.]], Function calls = 29900,             Best zeromode: [[-8.33606847e-01]
 [-5.24090540e-01]
 [-1.70857563e-01]
 [ 2.27752182e-02]
 [ 2.54454535e-02]
 [-6.89686302e-03]
 [-3.84555758e-03]
 [ 2.68820000e-03]
 [-8.51955240e-05]
 [-7.16919514e-04]
 [ 3.43347180e-04]
 [-3.68991893e-06]
 [-1.31194554e-04]
 [ 5.75798399e-05]
 [-1.92268539e-05]
 [-2.60867369e-05]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 6: Best fidelity = [[1.]], Function calls = 9077,             Best zeromode: [[ 8.33621446e-01]
 [ 5.24073221e-01]
 [ 1.70837964e-01]
 [-2.27846164e-02]
 [-2.54467362e-02]
 [ 6.89802060e-03]
 [ 3.84546918e-03]
 [-2.68823911e-03]
 [ 8.55857733e-05]
 [ 7.17199753e-04]
 [-3.43227772e-04]
 [ 3.73858249e-06]
 [ 1.31709452e-04]
 [-5.76732971e-05]
 [ 1.93466439e-05]
 [ 2.61639839e-05]]


### Print the results

In [27]:
# Print the results
print('The best zeromodes are:')
print(zeromodes_N_16)
print()

print('The performance metrics are:')
print(metrics_N_16)
print()

The best zeromodes are:
{'P_BFGS-RealAmplitudes': {1: array([[ 8.68420693e-01],
       [ 4.76332550e-01],
       [ 1.35730048e-01],
       [-1.77158933e-02],
       [-1.27885846e-02],
       [-6.60396073e-03],
       [-1.03448866e-03],
       [-2.49227119e-03],
       [-1.18448929e-03],
       [-6.49739010e-04],
       [-1.85227417e-04],
       [ 2.44416497e-05],
       [ 1.68231990e-05],
       [ 5.87262866e-06],
       [-5.24921159e-06],
       [ 2.21507288e-05]]), 2: array([[-8.33725855e-01],
       [-5.23932592e-01],
       [-1.70849425e-01],
       [ 2.24956470e-02],
       [ 2.51356012e-02],
       [-6.93386742e-03],
       [-3.77425122e-03],
       [ 2.43947790e-03],
       [-2.67408160e-04],
       [-3.09802272e-04],
       [-1.24729612e-04],
       [ 2.02900327e-05],
       [-6.18055473e-06],
       [ 1.78402823e-05],
       [ 8.03589957e-06],
       [-3.35809764e-06]]), 3: array([[ 8.33787016e-01],
       [ 5.23892640e-01],
       [ 1.70646521e-01],
       [-2.26737909e-02],


### Computing relative errors in <x^2>

In [30]:
# Get the zeromodes
zeromodes = [
    zeromode
    for depths in zeromodes_N_16.values()
    for zeromode in depths.values()
]

# Initialize
relative_errors = []

# Compute 
for zeromode in zeromodes:
    x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
    
    # Ensure quantum_expectation is a scalar
    quantum_expectation = compute_expectation_x_squared_simpson(x, y, 2)
    if isinstance(quantum_expectation, np.ndarray):
        quantum_expectation = quantum_expectation.item()  # Convert to scalar if necessary
    
    # Compute the relative error
    relative_error = compute_errors(classical_expectation, quantum_expectation)
    relative_errors.append(relative_error)

# Get the relative errors for the individual optimizers
errors_realamps = relative_errors[:6]
errors_twolocal = relative_errors[6:12]
errors_efficientsu2 = relative_errors[12:18]

## Print the relative errors
print('The errors for RealAmplitudes are:')
print(errors_realamps)
print()

print('The errors for TwoLocal are:')
print(errors_twolocal)
print()

print('The errors for EfficientSU2 are:')
print(errors_efficientsu2)
print()

C:\Users\Owner\AppData\Local\Temp\ipykernel_2564\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


The errors for RealAmplitudes are:
[0.10954897637113721, 0.0021880553592987452, 0.002599905505952389, 0.002675604318302532, 0.00024962496242818364, 2.2624137595374434e-05]

The errors for TwoLocal are:
[0.08860942813547215, 0.003006331171391604, 0.004528045888686157, 0.002887984779343588, 8.55893026406257e-05, 1.099962139495267e-05]

The errors for EfficientSU2 are:
[0.08085097810481134, 0.056198033626417905, 0.0005280706741107806, 6.598750025316014e-06, 3.794227361317616e-05, 1.0584783186409784e-05]



### For the pure Gaussian (b = 0) case

### For N = 2

### Get the matrix and the zeromode

In [20]:
## Get the matrix and zeromode
nmax = 3
a = 1
gamma = 1
dx = 0.01
c = 0
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode_classic = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode_classic)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode_classic, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[  6.75       -25.45584412]
 [-25.45584412 102.75      ]]

The zeromode is:
[[-0.97042573]
 [-0.24139987]]

The value of <x^2> is:
0.40008586593679



C:\Users\Owner\AppData\Local\Temp\ipykernel_8728\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


### Run VQE for multiple ansatz depths (fix at 5000 optimization iterations)

In [23]:
## Get the optimizers
maxiter = 5000
optimizer = P_BFGS(maxfun = maxiter)

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
max_depth = 6

# Compute 
zeromodes_N_2, metrics_N_2 = analyze_ansatz_performance(matrix, zeromode_classic, optimizer, num_qubits, max_depth)

For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.



Running VQE for optimizer-ansatz pair: P_BFGS-RealAmplitudes


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-RealAmplitudes - Depth 1: Best fidelity = [[1.]], Function calls = 18,             Best zeromode: [[-0.97042573]
 [-0.24139987]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


...

P_BFGS-EfficientSU2 - Depth 4: Best fidelity = [[1.]], Function calls = 54,             Best zeromode: [[0.97042573]
 [0.24139986]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 5: Best fidelity = [[1.]], Function calls = 49,             Best zeromode: [[0.97042573]
 [0.24139986]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 6: Best fidelity = [[1.]], Function calls = 72,             Best zeromode: [[0.97042573]
 [0.24139987]]


### Print the results

In [26]:
# Print the results
print('The best zeromodes are:')
print(zeromodes_N_2)
print()

print('The performance metrics are:')
print(metrics_N_2)
print()

The best zeromodes are:
{'P_BFGS-RealAmplitudes': {1: array([[-0.97042573],
       [-0.24139987]]), 2: array([[0.97042573],
       [0.24139988]]), 3: array([[0.97042573],
       [0.24139987]]), 4: array([[0.97042573],
       [0.24139986]]), 5: array([[0.97042573],
       [0.24139986]]), 6: array([[0.97042573],
       [0.24139987]])}, 'P_BFGS-TwoLocal': {1: array([[0.97042573],
       [0.24139986]]), 2: array([[0.97042573],
       [0.24139986]]), 3: array([[0.97042573],
       [0.24139987]]), 4: array([[-0.97042574],
       [-0.24139986]]), 5: array([[-0.97042573],
       [-0.24139987]]), 6: array([[-0.97042573],
       [-0.24139986]])}, 'P_BFGS-EfficientSU2': {1: array([[-0.97042573],
       [-0.24139987]]), 2: array([[0.97042573],
       [0.24139988]]), 3: array([[0.97042573],
       [0.24139987]]), 4: array([[0.97042573],
       [0.24139986]]), 5: array([[0.97042573],
       [0.24139986]]), 6: array([[0.97042573],
       [0.24139987]])}}

The performance metrics are:
{'P_BFGS-RealAmp

### Computing relative errors in <x^2>

In [29]:
# Get the zeromodes
zeromodes = [
    zeromode
    for depths in zeromodes_N_2.values()
    for zeromode in depths.values()
]

# Initialize
relative_errors = []

# Compute 
for zeromode in zeromodes:
    x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
    
    # Ensure quantum_expectation is a scalar
    quantum_expectation = compute_expectation_x_squared_simpson(x, y, 2)
    if isinstance(quantum_expectation, np.ndarray):
        quantum_expectation = quantum_expectation.item()  # Convert to scalar if necessary
    
    # Compute the relative error
    relative_error = compute_errors(classical_expectation, quantum_expectation)
    relative_errors.append(relative_error)

# Get the relative errors for the individual optimizers
errors_realamps = relative_errors[:6]
errors_twolocal = relative_errors[6:12]
errors_efficientsu2 = relative_errors[12:18]

## Print the relative errors
print('The errors for RealAmplitudes are:')
print(errors_realamps)
print()

print('The errors for TwoLocal are:')
print(errors_twolocal)
print()

print('The errors for EfficientSU2 are:')
print(errors_efficientsu2)
print()

C:\Users\Owner\AppData\Local\Temp\ipykernel_8728\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


The errors for RealAmplitudes are:
[3.6610356447967732e-09, 1.2055055698140791e-08, 4.158149946195491e-10, 6.934023673478102e-09, 3.0221256477785176e-09, 2.215123583926457e-09]

The errors for TwoLocal are:
[4.0738645029428386e-09, 6.00398604376116e-09, 8.20964839517545e-10, 1.2230960392636939e-08, 2.543102571574534e-09, 3.1474876029311467e-09]

The errors for EfficientSU2 are:
[3.6610356447967732e-09, 1.2055055698140791e-08, 4.158149946195491e-10, 6.934023673478102e-09, 3.0221256477785176e-09, 2.215123583926457e-09]



### For N = 4

### Get the matrix and the zeromode

In [32]:
## Get the matrix and zeromode
nmax = 7
a = 1
gamma = 1
dx = 0.01
c = 0
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode_classic = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode_classic)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode_classic, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[   6.75        -25.45584412   18.37117307    0.        ]
 [ -25.45584412  129.75       -173.20508076   71.15124735]
 [  18.37117307 -173.20508076  448.75       -449.13249715]
 [   0.           71.15124735 -449.13249715  837.75      ]]

The zeromode is:
[[-0.92504521]
 [-0.34807967]
 [-0.14437197]
 [-0.04783967]]

The value of <x^2> is:
0.6419765393615697



C:\Users\Owner\AppData\Local\Temp\ipykernel_8728\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


### Run VQE for multiple ansatz depths (fix at 5000 optimization iterations)

In [35]:
## Get the optimizers
maxiter = 5000
optimizer = P_BFGS(maxfun = maxiter)

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
max_depth = 6

# Compute 
zeromodes_N_4, metrics_N_4 = analyze_ansatz_performance(matrix, zeromode_classic, optimizer, num_qubits, max_depth)

For Windows, using only current process. Multiple core use not supported.



Running VQE for optimizer-ansatz pair: P_BFGS-RealAmplitudes


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-RealAmplitudes - Depth 1: Best fidelity = [[1.]], Function calls = 165,             Best zeromode: [[0.92504521]
 [0.34807967]
 [0.14437198]
 [0.04783968]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


...

P_BFGS-EfficientSU2 - Depth 4: Best fidelity = [[1.]], Function calls = 836,             Best zeromode: [[0.92504522]
 [0.34807965]
 [0.14437198]
 [0.04783967]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 5: Best fidelity = [[1.]], Function calls = 650,             Best zeromode: [[-0.92504521]
 [-0.34807968]
 [-0.14437197]
 [-0.04783967]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 6: Best fidelity = [[1.]], Function calls = 780,             Best zeromode: [[0.92504521]
 [0.34807968]
 [0.14437197]
 [0.04783967]]


### Print the results

In [38]:
# Print the results
print('The best zeromodes are:')
print(zeromodes_N_4)
print()

print('The performance metrics are:')
print(metrics_N_4)
print()

The best zeromodes are:
{'P_BFGS-RealAmplitudes': {1: array([[0.92504521],
       [0.34807967],
       [0.14437198],
       [0.04783968]]), 2: array([[-0.92504522],
       [-0.34807966],
       [-0.14437195],
       [-0.04783967]]), 3: array([[-0.92504521],
       [-0.34807966],
       [-0.14437198],
       [-0.04783967]]), 4: array([[0.9250452 ],
       [0.34807968],
       [0.14437198],
       [0.04783969]]), 5: array([[0.92504521],
       [0.34807968],
       [0.14437198],
       [0.04783968]]), 6: array([[0.92504522],
       [0.34807965],
       [0.14437196],
       [0.04783967]])}, 'P_BFGS-TwoLocal': {1: array([[-0.9250452 ],
       [-0.34807969],
       [-0.14437196],
       [-0.04783966]]), 2: array([[-0.92504522],
       [-0.34807966],
       [-0.14437196],
       [-0.04783966]]), 3: array([[-0.92504521],
       [-0.34807968],
       [-0.14437198],
       [-0.04783968]]), 4: array([[-0.92504521],
       [-0.34807967],
       [-0.14437197],
       [-0.04783967]]), 5: array([[0.9

### Computing relative errors in <x^2>

In [41]:
# Get the zeromodes
zeromodes = [
    zeromode
    for depths in zeromodes_N_4.values()
    for zeromode in depths.values()
]

# Initialize
relative_errors = []

# Compute 
for zeromode in zeromodes:
    x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
    
    # Ensure quantum_expectation is a scalar
    quantum_expectation = compute_expectation_x_squared_simpson(x, y, 2)
    if isinstance(quantum_expectation, np.ndarray):
        quantum_expectation = quantum_expectation.item()  # Convert to scalar if necessary
    
    # Compute the relative error
    relative_error = compute_errors(classical_expectation, quantum_expectation)
    relative_errors.append(relative_error)

# Get the relative errors for the individual optimizers
errors_realamps = relative_errors[:6]
errors_twolocal = relative_errors[6:12]
errors_efficientsu2 = relative_errors[12:18]

## Print the relative errors
print('The errors for RealAmplitudes are:')
print(errors_realamps)
print()

print('The errors for TwoLocal are:')
print(errors_twolocal)
print()

print('The errors for EfficientSU2 are:')
print(errors_efficientsu2)
print()

C:\Users\Owner\AppData\Local\Temp\ipykernel_8728\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


The errors for RealAmplitudes are:
[2.793487837188261e-08, 3.30853213315141e-08, 1.2306099970677197e-08, 5.298089029417922e-08, 3.376985487108626e-08, 1.5065396410290868e-08]

The errors for TwoLocal are:
[2.269803009897159e-09, 2.2078812052482782e-08, 4.2637409509773645e-08, 1.5125560245696287e-09, 1.1568859866193096e-08, 3.447494523515272e-08]

The errors for EfficientSU2 are:
[2.793487837188261e-08, 1.4856949262217022e-08, 1.3152380239007111e-08, 4.209709696326017e-09, 1.3267215390839426e-08, 1.4718709165768956e-08]



### For N = 8

### Get the matrix and the zeromode

In [21]:
## Get the matrix and zeromode
nmax = 15
a = 1
gamma = 1
dx = 0.01
c = 0
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode_classic = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode_classic)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode_classic, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[    6.75         -25.45584412    18.37117307     0.
      0.             0.             0.             0.        ]
 [  -25.45584412   129.75        -173.20508076    71.15124735
      0.             0.             0.             0.        ]
 [   18.37117307  -173.20508076   448.75        -449.13249715
    153.70426149     0.             0.             0.        ]
 [    0.            71.15124735  -449.13249715   963.75
   -853.09788418   266.22359024     0.             0.        ]
 [    0.             0.           153.70426149  -853.09788418
   1674.75       -1385.07761515   408.7327978      0.        ]
 [    0.             0.             0.           266.22359024
  -1385.07761515  2581.75       -2045.06430217   581.2379031 ]
 [    0.             0.             0.             0.
    408.7327978  -2045.06430217  3684.75       -2833.05488828]
 [    0.             0.             0.             0.
      0.           581.2379031  -2833.05488828  4443.75      ]]

The zeromode 

C:\Users\Owner\AppData\Local\Temp\ipykernel_2916\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


### Run VQE for multiple ansatz depths (fix at 5000 optimization iterations)

In [24]:
## Get the optimizers
maxiter = 5000
optimizer = P_BFGS(maxfun = maxiter)

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
max_depth = 6

# Compute 
zeromodes_N_8, metrics_N_8 = analyze_ansatz_performance(matrix, zeromode_classic, optimizer, num_qubits, max_depth)

For Windows, using only current process. Multiple core use not supported.



Running VQE for optimizer-ansatz pair: P_BFGS-RealAmplitudes


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-RealAmplitudes - Depth 1: Best fidelity = [[0.99974471]], Function calls = 1281,             Best zeromode: [[0.90192158]
 [0.37483827]
 [0.186312  ]
 [0.09402016]
 [0.04536018]
 [0.01918989]
 [0.00720939]
 [0.00200087]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


...

P_BFGS-EfficientSU2 - Depth 4: Best fidelity = [[1.]], Function calls = 2800,             Best zeromode: [[0.89865022]
 [0.37648519]
 [0.19152282]
 [0.10119186]
 [0.05336692]
 [0.02717775]
 [0.01261408]
 [0.00448713]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 5: Best fidelity = [[1.]], Function calls = 874,             Best zeromode: [[-0.89865016]
 [-0.37648529]
 [-0.19152288]
 [-0.10119188]
 [-0.0533669 ]
 [-0.02717771]
 [-0.01261404]
 [-0.00448709]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 6: Best fidelity = [[1.]], Function calls = 1496,             Best zeromode: [[-0.89865007]
 [-0.37648546]
 [-0.19152295]
 [-0.10119186]
 [-0.05336691]
 [-0.02717777]
 [-0.01261408]
 [-0.00448712]]


### Print the results

In [27]:
# Print the results
print('The best zeromodes are:')
print(zeromodes_N_8)
print()

print('The performance metrics are:')
print(metrics_N_8)
print()

The best zeromodes are:
{'P_BFGS-RealAmplitudes': {1: array([[0.90192158],
       [0.37483827],
       [0.186312  ],
       [0.09402016],
       [0.04536018],
       [0.01918989],
       [0.00720939],
       [0.00200087]]), 2: array([[0.89865003],
       [0.37648613],
       [0.1915232 ],
       [0.1011912 ],
       [0.05336528],
       [0.02717535],
       [0.01261162],
       [0.00448577]]), 3: array([[-0.89864988],
       [-0.3764857 ],
       [-0.19152321],
       [-0.10119214],
       [-0.05336707],
       [-0.02717783],
       [-0.01261409],
       [-0.0044871 ]]), 4: array([[0.89865019],
       [0.37648526],
       [0.1915228 ],
       [0.10119183],
       [0.05336692],
       [0.02717782],
       [0.01261413],
       [0.00448715]]), 5: array([[0.89865027],
       [0.37648517],
       [0.19152267],
       [0.10119173],
       [0.05336687],
       [0.02717776],
       [0.01261409],
       [0.00448713]]), 6: array([[0.89865021],
       [0.37648522],
       [0.19152275],
       [0.

### Computing relative errors in <x^2>

In [30]:
# Get the zeromodes
zeromodes = [
    zeromode
    for depths in zeromodes_N_8.values()
    for zeromode in depths.values()
]

# Initialize
relative_errors = []

# Compute 
for zeromode in zeromodes:
    x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
    
    # Ensure quantum_expectation is a scalar
    quantum_expectation = compute_expectation_x_squared_simpson(x, y, 2)
    if isinstance(quantum_expectation, np.ndarray):
        quantum_expectation = quantum_expectation.item()  # Convert to scalar if necessary
    
    # Compute the relative error
    relative_error = compute_errors(classical_expectation, quantum_expectation)
    relative_errors.append(relative_error)

# Get the relative errors for the individual optimizers
errors_realamps = relative_errors[:6]
errors_twolocal = relative_errors[6:12]
errors_efficientsu2 = relative_errors[12:18]

## Print the relative errors
print('The errors for RealAmplitudes are:')
print(errors_realamps)
print()

print('The errors for TwoLocal are:')
print(errors_twolocal)
print()

print('The errors for EfficientSU2 are:')
print(errors_efficientsu2)
print()

C:\Users\Owner\AppData\Local\Temp\ipykernel_2916\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


The errors for RealAmplitudes are:
[0.055628580649997435, 1.4815925644047548e-05, 1.1472408116226784e-06, 2.5766149873513357e-07, 3.0840543854484014e-07, 3.100925051475348e-07]

The errors for TwoLocal are:
[0.0556268317128886, 1.20060066981278e-06, 5.214487040490026e-08, 1.3364593573959302e-07, 4.8268438362489556e-08, 3.021334600993426e-07]

The errors for EfficientSU2 are:
[0.05562852262162045, 5.565939387907108e-08, 1.696055196625737e-07, 2.0244240144136225e-08, 1.6407277395458455e-07, 2.3354812012511053e-07]



### For N = 16

### Get the matrix and the zeromode

In [19]:
## Get the matrix and zeromode
nmax = 31
a = 1
gamma = 1
dx = 0.01
c = 0
L = 1/2
x = np.linspace(-4, 4, int(8/dx))

# Matrix and zeromode
matrix, zeromode_classic = get_zeromode(nmax, a, c, L, gamma)

## Print the matrix and zeromode
print('The matrix is:')
print(matrix)
print()

# Print the zeromode
print('The zeromode is:')
print(zeromode_classic)
print()

## Compute the classical <x^2> value
x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode_classic, normalize = True, make_even = True)
classical_expectation = compute_expectation_x_squared_simpson(x, y, 2)

# Print the classical expectation value
print('The value of <x^2> is:')
print(classical_expectation)
print()

The matrix is:
[[ 6.75000000e+00 -2.54558441e+01  1.83711731e+01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-2.54558441e+01  1.29750000e+02 -1.73205081e+02  7.11512474e+01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.83711731e+01 -1.73205081e+02  4.48750000e+02 -4.49132497e+02
   1.53704261e+02  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  7.11512474e+01 -4.49132497e+02  9.63750000e+02
  -8.53097884e+02  2.66223590e+02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000

C:\Users\Owner\AppData\Local\Temp\ipykernel_26752\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


The value of <x^2> is:
0.996520371963445



### Run VQE for multiple ansatz depths (fix at 50_000 optimization iterations)

### Note: we consider 6 independent VQE runs at each ansatz depth

In [22]:
## Get the optimizers
maxiter = 50_000
optimizer = P_BFGS(maxfun = maxiter)

# Define parameters
dimension = matrix.shape[0]
num_qubits = int(np.log2(dimension))
max_depth = 6

# Compute 
zeromodes_N_16, metrics_N_16 = analyze_ansatz_performance(matrix, zeromode_classic, optimizer, num_qubits, max_depth)

For Windows, using only current process. Multiple core use not supported.



Running VQE for optimizer-ansatz pair: P_BFGS-RealAmplitudes


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-RealAmplitudes - Depth 1: Best fidelity = [[0.99869519]], Function calls = 2466,             Best zeromode: [[ 9.02920421e-01]
 [ 3.73568851e-01]
 [ 1.84870251e-01]
 [ 9.28299877e-02]
 [ 4.45524259e-02]
 [ 1.88014830e-02]
 [ 6.75720753e-03]
 [ 1.59261470e-03]
 [ 1.39618818e-06]
 [ 5.72884199e-07]
 [ 3.16441034e-07]
 [ 1.82175270e-07]
 [ 1.52926527e-07]
 [ 1.61131877e-07]
 [-5.96467102e-07]
 [-7.77468027e-07]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


...

P_BFGS-EfficientSU2 - Depth 4: Best fidelity = [[0.99999994]], Function calls = 29421,             Best zeromode: [[-8.94447885e-01]
 [-3.79460343e-01]
 [-1.97154092e-01]
 [-1.07968098e-01]
 [-6.05814991e-02]
 [-3.44690160e-02]
 [-1.97870886e-02]
 [-1.14258318e-02]
 [-6.61759232e-03]
 [-3.83402437e-03]
 [-2.21351722e-03]
 [-1.26504828e-03]
 [-7.07395348e-04]
 [-3.78212353e-04]
 [-1.83434005e-04]
 [-6.79286276e-05]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 5: Best fidelity = [[0.99999997]], Function calls = 33525,             Best zeromode: [[8.94456745e-01]
 [3.79456061e-01]
 [1.97143380e-01]
 [1.07952720e-01]
 [6.05655761e-02]
 [3.44532785e-02]
 [1.97669766e-02]
 [1.14010125e-02]
 [6.59236753e-03]
 [3.81037743e-03]
 [2.19241672e-03]
 [1.24754540e-03]
 [6.93878716e-04]
 [3.68652287e-04]
 [1.77487417e-04]
 [6.52777500e-05]]


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 6: Best fidelity = [[1.]], Function calls = 9715,             Best zeromode: [[-8.94494039e-01]
 [-3.79430394e-01]
 [-1.97095900e-01]
 [-1.07897487e-01]
 [-6.05042550e-02]
 [-3.43890470e-02]
 [-1.97062875e-02]
 [-1.13464771e-02]
 [-6.54587242e-03]
 [-3.77220361e-03]
 [-2.16249136e-03]
 [-1.22513851e-03]
 [-6.78055423e-04]
 [-3.58344960e-04]
 [-1.71594367e-04]
 [-6.28081071e-05]]


### Print the results

In [25]:
# Print the results
print('The best zeromodes are:')
print(zeromodes_N_16)
print()

print('The performance metrics are:')
print(metrics_N_16)
print()

The best zeromodes are:
{'P_BFGS-RealAmplitudes': {1: array([[ 9.02920421e-01],
       [ 3.73568851e-01],
       [ 1.84870251e-01],
       [ 9.28299877e-02],
       [ 4.45524259e-02],
       [ 1.88014830e-02],
       [ 6.75720753e-03],
       [ 1.59261470e-03],
       [ 1.39618818e-06],
       [ 5.72884199e-07],
       [ 3.16441034e-07],
       [ 1.82175270e-07],
       [ 1.52926527e-07],
       [ 1.61131877e-07],
       [-5.96467102e-07],
       [-7.77468027e-07]]), 2: array([[-8.96118512e-01],
       [-3.78353244e-01],
       [-1.94954665e-01],
       [-1.05293865e-01],
       [-5.77428609e-02],
       [-3.16305329e-02],
       [-1.70458867e-02],
       [-8.85308161e-03],
       [-4.28389071e-03],
       [-1.80728176e-03],
       [-8.24396355e-04],
       [-4.92574301e-04],
       [-2.75624822e-04],
       [-1.51048043e-04],
       [-8.26009955e-05],
       [-4.16054427e-05]]), 3: array([[-8.95416282e-01],
       [-3.78866680e-01],
       [-1.95835618e-01],
       [-1.06371408e-01],


### Computing relative errors in <x^2>

In [28]:
# Get the zeromodes
zeromodes = [
    zeromode
    for depths in zeromodes_N_16.values()
    for zeromode in depths.values()
]

# Initialize
relative_errors = []

# Compute 
for zeromode in zeromodes:
    x, y = get_pdf(nmax, x, dx, L, shift = 0, zeromode_qpe = zeromode, normalize = True, make_even = True)
    
    # Ensure quantum_expectation is a scalar
    quantum_expectation = compute_expectation_x_squared_simpson(x, y, 2)
    if isinstance(quantum_expectation, np.ndarray):
        quantum_expectation = quantum_expectation.item()  # Convert to scalar if necessary
    
    # Compute the relative error
    relative_error = compute_errors(classical_expectation, quantum_expectation)
    relative_errors.append(relative_error)

# Get the relative errors for the individual optimizers
errors_realamps = relative_errors[:6]
errors_twolocal = relative_errors[6:12]
errors_efficientsu2 = relative_errors[12:18]

## Print the relative errors
print('The errors for RealAmplitudes are:')
print(errors_realamps)
print()

print('The errors for TwoLocal are:')
print(errors_twolocal)
print()

print('The errors for EfficientSU2 are:')
print(errors_efficientsu2)
print()

C:\Users\Owner\AppData\Local\Temp\ipykernel_26752\795971042.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eigenvector[2*i] = eigenvector_old[i]


The errors for RealAmplitudes are:
[0.15389594872887838, 0.04033047569940305, 0.02118097223352969, 0.005634602472894297, 0.001781923171440637, 0.00010669664181539861]

The errors for TwoLocal are:
[0.15387903029169145, 0.038408860053955864, 0.006745303154615376, 0.000812757892764334, 0.00019465265500145218, 1.134207382746479e-05]

The errors for EfficientSU2 are:
[50.85809962284226, 0.4072691342736743, 0.029694819112357124, 0.001483457455359662, 0.0010002596092744593, 5.0016590804608324e-05]

